In [85]:
import pyspark
from pyspark.sql import SparkSession
from IPython.display import display
from pyspark.sql.functions import udf ,col ,split 
from pyspark.sql.types import DateType,StringType
from datetime import datetime

# Basic operations

In [69]:
spark = SparkSession.builder.appName("stocksly").getOrCreate()
spark

In [70]:
stocks = spark.read.csv(
    "../data_sourcing/source/stocks.csv", 
    header=True, 
    inferSchema=True
)

In [71]:
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|
+---+---------+-------------------+-----+-----+-----+-----+------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|
+---+---------+-------------------+-----+-----+-----+-----+------+
only showing top 5 rows



25/02/16 02:46:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [72]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- stockname: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)



In [73]:
stocks.select("stockname").show(5)

+---------+
|stockname|
+---------+
|    PLTR |
|    PLTR |
|    PLTR |
|    PLTR |
|    PLTR |
+---------+
only showing top 5 rows



In [74]:
stocks.select(
    ["stockname","open", "close"]
).show(5)

+---------+-----+-----+
|stockname| open|close|
+---------+-----+-----+
|    PLTR |81.94|81.44|
|    PLTR |81.39|82.01|
|    PLTR |81.96|82.13|
|    PLTR |82.19|82.38|
|    PLTR | 82.4|82.32|
+---------+-----+-----+
only showing top 5 rows



In [75]:
stocks.filter(
    stocks.stockname == "TSLA"
).show(5)

25/02/16 02:46:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+------+---------+-------------------+------+------+------+------+------+
|   _c0|stockname|          timestamp|  open|  high|   low| close|volume|
+------+---------+-------------------+------+------+------+------+------+
|354692|     TSLA|2024-12-23 09:00:00| 431.6| 434.2|431.28|434.19|  NULL|
|354693|     TSLA|2024-12-23 09:01:00|434.19|435.13|433.53| 434.5|  NULL|
|354694|     TSLA|2024-12-23 09:02:00|434.63|434.86|433.02| 433.7|  NULL|
|354695|     TSLA|2024-12-23 09:03:00|433.69|433.97|433.29| 433.6|  NULL|
|354696|     TSLA|2024-12-23 09:04:00|433.57| 433.6|432.61|433.28|  NULL|
+------+---------+-------------------+------+------+------+------+------+
only showing top 5 rows



In [76]:
stocks.filter(
    (
        (stocks.stockname == "TSLA") |
        (stocks.stockname == "PLTR")
    ) & (
    stocks.timestamp == "2024-12-23 09:00:00")
).show()

25/02/16 02:46:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+-------+---------+-------------------+-----+-----+------+------+------+
|    _c0|stockname|          timestamp| open| high|   low| close|volume|
+-------+---------+-------------------+-----+-----+------+------+------+
| 354692|     TSLA|2024-12-23 09:00:00|431.6|434.2|431.28|434.19|  NULL|
|5149233|     PLTR|2024-12-23 09:00:00|81.94|81.95| 81.16| 81.44|   0.0|
+-------+---------+-------------------+-----+-----+------+------+------+



In [77]:
stocks.filter(
    stocks.stockname.isin("TSLA", "PLTR") &
    (stocks.timestamp == "2024-12-23 09:00:00")
).show()

25/02/16 02:47:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+-------+---------+-------------------+-----+-----+------+------+------+
|    _c0|stockname|          timestamp| open| high|   low| close|volume|
+-------+---------+-------------------+-----+-----+------+------+------+
| 354692|     TSLA|2024-12-23 09:00:00|431.6|434.2|431.28|434.19|  NULL|
|5149233|     PLTR|2024-12-23 09:00:00|81.94|81.95| 81.16| 81.44|   0.0|
+-------+---------+-------------------+-----+-----+------+------+------+



In [78]:
stocks = stocks.withColumn(
    "date", split(
        col("timestamp"), 
    " ")[0]
    )
stocks = stocks.withColumn(
    "time", split(
        col("timestamp"), 
    " ")[1]
    )
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
only showing top 5 rows



25/02/16 02:48:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [79]:
date_parser = udf(
    lambda date : datetime.strptime(date, "%Y-%m-%d"),DateType()
)

In [80]:
stocks = stocks.withColumn(
    "new_date",
    date_parser(stocks.date)  
)

stocks.show(5)

25/02/16 02:48:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



In [81]:
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/16 02:48:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [86]:
def upper_turner(stockname):
    return stockname.upper()

upper_turner = udf(
    upper_turner, StringType()
)

In [87]:
stocks = stocks.withColumn(
    "stockname",upper_turner(stocks.stockname)
)
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/16 03:02:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [88]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- stockname: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- new_date: date (nullable = true)



In [89]:
new_stocks = stocks.select(
    "stockname",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "new_date",
    "time"
)

In [90]:
new_stocks.show(5)

+---------+-----+-----+-----+-----+------+----------+--------+
|stockname| open| high|  low|close|volume|  new_date|    time|
+---------+-----+-----+-----+-----+------+----------+--------+
|    PLTR |81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|
|    PLTR |81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|
|    PLTR |81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|
|    PLTR |82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|
|    PLTR | 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|
+---------+-----+-----+-----+-----+------+----------+--------+
only showing top 5 rows



In [92]:
new_stocks.describe().show()

25/02/16 03:11:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+------------------+------------------+------------------+------------------+-----------------+--------+
|summary|stockname|              open|              high|               low|             close|           volume|    time|
+-------+---------+------------------+------------------+------------------+------------------+-----------------+--------+
|  count| 69242798|          63772250|          63772250|          63772250|          63772250|         60931060|69242798|
|   mean|     NULL|157.51115919261184|157.55508494635941| 157.4645094963356|157.51097045883867|55927.47375937658|    NULL|
| stddev|     NULL|3076.7758426276955|3076.7765233488894| 3076.775057143922|3076.7758180089277|7578621.679077058|    NULL|
|    min|      AAL| 0.002199999988079| 0.002199999988079|0.0020999999251216|0.0020999999251216|              0.0|00:00:00|
|    max|      ZK |     109110.390625|     109110.390625|     109110.390625|     109110.390625|   4.590406656E10|23:59:10|
+-------+-------